In [2]:
# Copyright Shawn Gu, 2016
from math import pi
import pandas as pd

from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Rect, HoverTool, Range1d, LinearAxis, WheelZoomTool, PanTool, ResetTool, ResizeTool, PreviewSaveTool

output_notebook()

ImportError: No module named 'pandas'

In [ ]:



def create_data_source(data_frame):
    '''
    Reference here: https://github.com/bokeh/bokeh/issues/1239
    '''
    return ColumnDataSource(
        data=dict(
            date=[x.strftime("%Y-%m-%d") for x in data_frame['date']],
            open=data_frame['open'],
            close=data_frame['close'],
            high=data_frame['high'],
            low=data_frame['low'],
        )
    )


def ricequant_candlestick_volume_plot(MD_df, plot_title=None):
    '''
    Copyright Shawn Gu, 2016
    '''
    input_df = pd.DataFrame(
        {'date': MD_df.index,
         'open': MD_df['OpeningPx'],
         'close': MD_df['ClosingPx'],
         'high': MD_df['HighPx'],
         'low': MD_df['LowPx'],
         'volume': MD_df['TotalVolumeTraded']}
    )
    candlestick_volume_plot(input_df, plot_title)
    

def candlestick_volume_plot(input_df, plot_title=None):
    '''
    Copyright Shawn Gu, 2016
    The code below is modified based on the snippet from http://bokeh.pydata.org/en/0.11.1/docs/gallery/candlestick.html.
    '''
    px_mids = (input_df['open'] + input_df['close']) / 2.0
    px_spans = abs(input_df['close'] - input_df['open'])
    
    vol_mids = input_df['volume'] / 2.0
    vol_spans = input_df['volume']
    max_vol = max(input_df['volume'])
    
    inc = input_df['close'] >= input_df['open']
    dec = input_df['open'] > input_df['close']
    
    inc_color = 'red'
    dec_color = 'green'
    
    width = 12*60*60*1000*1.2 # in ms
    
    ht = HoverTool(
    tooltips=[
            ("date", "@date"),
            ("open", "@open"),
            ("close", "@close"),
            ("high", "@high"),
            ("low", "@low"),
        ]
    )
    TOOLS = [ht, WheelZoomTool(dimensions=['width']), ResizeTool(), ResetTool(),
             PanTool(dimensions=['width']), PreviewSaveTool()]
    
    max_px = max(input_df['high'])
    min_px = min(input_df['low'])
    
    px_range = max_px - min_px
    
    primary_y_range = (min_px - px_range / 2.0, max_px + px_range * 0.1)
    
    p = figure(x_axis_type="datetime", tools=TOOLS, plot_height=600, plot_width=950,
               toolbar_location="above", y_range=primary_y_range)
    
    if plot_title:
        p.title = plot_title
    
    p.xaxis.major_label_orientation = pi/4
    p.grid.grid_line_alpha=0.3
    p.background_fill = "black"
    
    
    p.extra_y_ranges = {"volume": Range1d(start=0, end=max_vol * 5)}
    p.add_layout(LinearAxis(y_range_name="volume"), 'right')

    px_rect_inc_src = create_data_source(input_df[inc])
    px_rect_dec_src = create_data_source(input_df[dec])

    p.segment(input_df.date[inc], input_df.high[inc], input_df.date[inc], input_df.low[inc], color=inc_color)
    p.segment(input_df.date[dec], input_df.high[dec], input_df.date[dec], input_df.low[dec], color=dec_color)
    p.rect(input_df.date[inc], px_mids[inc], width, px_spans[inc],
           fill_color=inc_color, line_color=inc_color, source=px_rect_inc_src)
    p.rect(input_df.date[dec], px_mids[dec], width, px_spans[dec],
           fill_color=dec_color, line_color=dec_color, source=px_rect_dec_src)

    p.rect(input_df.date[inc], vol_mids[inc], width, vol_spans[inc],
             fill_color=inc_color, color=inc_color, y_range_name="volume")
    p.rect(input_df.date[dec], vol_mids[dec], width, vol_spans[dec],
             fill_color=dec_color, color=dec_color, y_range_name="volume")

    show(p)

In [ ]:
# 使用范例
hf_df = get_price('600207.XSHG', start_date='2013-01-02', end_date='2013-05-15')
ricequant_candlestick_volume_plot(hf_df)